# 2025 Data Scrape (Infosys API)

In [14]:
import pandas as pd
from infotennis.scrapers.scraping_functions_atp import scrape_ATP_tournament, scrape_ATP_calendar
from infotennis.scrapers.scrape_match_data import scrape_ATP_match_data
from rapidfuzz import process, fuzz
from tqdm import tqdm
import time
import numpy as np
import re
from json import JSONDecodeError
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Tournaments Calendar

In [15]:
year = 2025
df_tournaments = scrape_ATP_calendar(year)
df_tournaments[['start', 'end']] = df_tournaments['date_start'].str.split(' - ', expand=True)


In [22]:
len(df_tournaments)

66

In [19]:
df_tournaments.loc[df_tournaments['tournament'] == 'Australian Open']


,year,tournament,tournament_id,category,location,date_start,tournament_status,draw,surface,finance,winner,url,start,end
5,2025,Australian Open,580,Grand Slam,"Melbourne, Australia",2025.01.01 - 2025.01.26,Completed,SGL 128 DBL 64,Outdoor Hard,"A$43,250,000","SGL: Jannik Sinner, DBL: Harri Heliovaara Henry Patten",https://www.atptour.com/en/scores/archive/australian-open/580/2025/results,2025.01.01,2025.01.26


### Matches Calendar
1. Scrape data for all matches in 2025

In [20]:
all_matches = []
for _, tourn in df_tournaments.iterrows():
    # If the “Results” URL is blank, skip
    if not tourn.url:
        continue
    df_matches = scrape_ATP_tournament(
        url=tourn.url,
        tournament=tourn.tournament,
        tournament_id=tourn.tournament_id,
        year=year,
        format="S"
    )
    all_matches.append(df_matches)

matches = pd.concat(all_matches, ignore_index=True)


In [21]:
matches.head()

,year,tournament,tournament_id,round,player1_name,player1_id,player1_seed,player1_nation,player2_name,player2_id,player2_seed,player2_nation,score,url,court_vision
0,2025,United Cup,9900,Final,Taylor Fritz,fb98,1,usa,Hubert Hurkacz,hb71,2,pol,64 57 76(4),https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms011,0
1,2025,United Cup,9900,Final,Coco Gauff,,1,usa,Iga Swiatek,,2,pol,64 64,,0
2,2025,United Cup,9900,Semifinals,Taylor Fritz,fb98,1,usa,Tomas Machac,m0fh,8,cze,67(4) 65,https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms021,0
3,2025,United Cup,9900,Semifinals,Coco Gauff,,1,usa,Karolina Muchova,,8,cze,61 64,,0
4,2025,United Cup,9900,Semifinals,"Desirae Krawczyk, Denis Kudla","-, kb09",1,"usa, usa","Gabriela Knutson, Patrik Rikl","-, rh13",8,"cze, cze",75 60,,0


2. Clean Data 

In [4]:
matches_2025 = matches.copy()

# Extract match id's from urls
matches_2025['match_id'] = matches_2025['url'].str.split('/').str[-1]

# drop duplicate match entry
mask = (matches_2025['tournament_id'] == '321') & (matches_2025['match_id'] == 'ms013')
idx_to_drop = matches_2025[mask].index[:1]
matches_2025 = matches_2025.drop(idx_to_drop)

# drop doubles & womens matches
matches_2025 = matches_2025[~matches_2025['player1_name'].str.contains(',', na=False)]
matches_2025['player1_id'].replace('', np.nan, inplace=True)
matches_2025['player2_id'].replace('', np.nan, inplace=True)
matches_2025 = matches_2025.dropna(subset=['player1_id', 'player2_id'])

# rename infosys player id's for clarity
matches_2025 = matches_2025.rename(columns={
    'player1_id': 'player1_sysid',
    'player2_id': 'player2_sysid',
})

2. Extract Tournament Info form Tournament Calendar

In [5]:
matches_2025['tournament_id'] = matches_2025['tournament_id'].astype(str)
df_tournaments['tournament_id'] = df_tournaments['tournament_id'].astype(str)

# merge dataframes for relevant columns
matches_2025 = pd.merge(matches_2025, df_tournaments[['tournament_id', 'start', 'end', 'category', 'draw', 'surface']], on='tournament_id', how='left')

# reformat
matches_2025['draw'] = matches_2025['draw'].str.extract(r'SGL\s*(\d+)', expand=False).astype(float)
matches_2025 = matches_2025.rename(columns={'start': 'tourney_date'})
matches_2025['surface'] = matches_2025['surface'].replace({
    'Outdoor Hard': 'Hard',
    'Outdoor Clay': 'Clay',
    'Outdoor Grass': 'Grass',
    'Indoor Hard': 'Hard',
    'Indoor ' : 'Hard'
})
matches_2025.head()

,year,tournament,tournament_id,round,player1_name,player1_sysid,player1_seed,player1_nation,player2_name,player2_sysid,player2_seed,player2_nation,score,url,court_vision,match_id,tourney_date,end,category,draw,surface
0,2025,United Cup,9900,Final,Taylor Fritz,fb98,1,usa,Hubert Hurkacz,hb71,2,pol,64 57 76(4),https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms011,0,ms011,2024.12.27,2025.01.05,Other,18.0,Hard
1,2025,United Cup,9900,Semifinals,Taylor Fritz,fb98,1,usa,Tomas Machac,m0fh,8,cze,67(4) 65,https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms021,0,ms021,2024.12.27,2025.01.05,Other,18.0,Hard
2,2025,United Cup,9900,Semifinals,Hubert Hurkacz,hb71,2,pol,Alexander Shevchenko,s0h2,9,kaz,63 62,https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms031,0,ms031,2024.12.27,2025.01.05,Other,18.0,Hard
3,2025,United Cup,9900,Quarterfinals,Tomas Machac,m0fh,8,cze,Flavio Cobolli,c0e9,4,ita,61 62,https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms051,0,ms051,2024.12.27,2025.01.05,Other,18.0,Hard
4,2025,United Cup,9900,Quarterfinals,Hubert Hurkacz,hb71,2,pol,Billy Harris,hd68,6,gbr,76(3) 75,https://www.atptour.com/en/scores/match-stats/archive/2025/9900/ms071,0,ms071,2024.12.27,2025.01.05,Other,18.0,Hard


### Assign Player Information
1. Player ID's assigned in alignment with Jeff Sackmann Dataset

In [6]:
# Load & normalize players lookup
players = pd.read_csv("./data/players/atp_players.csv")
players['first_lc'] = players['name_first'].str.lower().str.strip()
players['last_lc']  = players['name_last'] .str.lower().str.strip()
players['full_lc']  = players['first_lc'] + ' ' + players['last_lc']

# Deduplicate
players_unique = players.drop_duplicates(subset='full_lc', keep='first')
id_map = dict(zip(players_unique['full_lc'], players_unique['player_id']))

# Map onto matches dataframe
m = matches_2025.copy()
m['p1_full_lc'] = m['player1_name'].str.lower().str.strip()
m['p2_full_lc'] = m['player2_name'].str.lower().str.strip()

m['player1_id'] = m['p1_full_lc'].map(id_map)
m['player2_id'] = m['p2_full_lc'].map(id_map)

# fuzzy fallback
choices = list(id_map.keys())
def fuzzy_id(name):
    if pd.isna(name): return None
    res = process.extractOne(name, choices, scorer=fuzz.QRatio, score_cutoff=85)
    return id_map[res[0]] if res else None

mask1 = m['player1_id'].isna()
m.loc[mask1, 'player1_id'] = m.loc[mask1, 'p1_full_lc'].apply(fuzzy_id)

mask2 = m['player2_id'].isna()
m.loc[mask2, 'player2_id'] = m.loc[mask2, 'p2_full_lc'].apply(fuzzy_id)

# Manual input for remaining missing IDs
unmatched_p1 = (
    m.loc[m['player1_id'].isna(), 'p1_full_lc']
     .drop_duplicates()
     .tolist()
)
for name in unmatched_p1:
    pid = input(f"Enter player_id for P1 name '{name}': ").strip()
    m.loc[m['p1_full_lc'] == name, 'player1_id'] = pid or None

unmatched_p2 = (
    m.loc[m['player2_id'].isna(), 'p2_full_lc']
     .drop_duplicates()
     .tolist()
)
for name in unmatched_p2:
    pid = input(f"Enter player_id for P2 name '{name}': ").strip()
    m.loc[m['p2_full_lc'] == name, 'player2_id'] = pid or None

# Clean up
m = m.drop(columns=['p1_full_lc','p2_full_lc'])
matches_2025 = m

2. Height and Age Data - Lookups from Jeff Sackmann Players Dataset

In [7]:
height_map = players.set_index('player_id')['height']
dob_map    = players.set_index('player_id')['dob']

# Map into matches dataframe
matches_2025['player1_height'] = matches_2025['player1_id'].map(height_map)
matches_2025['player1_dob']    = matches_2025['player1_id'].map(dob_map)

matches_2025['player2_height'] = matches_2025['player2_id'].map(height_map)
matches_2025['player2_dob']    = matches_2025['player2_id'].map(dob_map)

matches_2025['player1_dob'] = matches_2025['player1_dob'].astype('Int64')  # pandas nullable int
matches_2025['player2_dob'] = matches_2025['player2_dob'].astype('Int64')  # pandas nullable int

# Calculate Age at time of match
matches_2025['player1_dob'] = pd.to_datetime(
    matches_2025['player1_dob'].astype(str),
    format='%Y%m%d', errors='coerce'
)
matches_2025['player2_dob'] = pd.to_datetime(
    matches_2025['player2_dob'].astype(str),
    format='%Y%m%d', errors='coerce'
)

matches_2025['tourney_date'] = pd.to_datetime(
    matches_2025['tourney_date'], errors='coerce')

matches_2025['player1_age'] = (
    (matches_2025['tourney_date'] - matches_2025['player1_dob'])
     .dt.days
    / 365.25
)
matches_2025['player2_age'] = (
    (matches_2025['tourney_date'] - matches_2025['player2_dob'])
     .dt.days
    / 365.25
)

matches_2025[['player1_age','player2_age']] = (
    matches_2025[['player1_age','player2_age']]
      .round(1)
)

# Clean Up
matches_2025.drop(columns=['player1_dob','player2_dob'], inplace=True)

### Standardize Dataframe formatting
1. Round, Tournament Level, Best of #Sets, and Column names refactor 

In [8]:
matches_2025['round'] = matches_2025['round'].replace({
    'Quarterfinals': 'QF',
    'Semifinals': 'SF',
    'Final': 'F',
    'Round Of 16': 'R16',
    'Round Of 32': 'R32',
    'Round Of 64': 'R64',
    'Round Of 128': 'R128',
    '1st Round Qualifying': 'ER',
    '2nd Round Qualifying': 'ER',
    '3rd Round Qualifying': 'ER',
    'Round Robin': 'RR',
    'Round Robin Day 2': 'RR',
    'Round Robin Day 3': 'RR',
    'Round Robin Day 4': 'RR',
    'Round Robin Day 5': 'RR',
    'Round Robin Day 6': 'RR',
})

matches_2025['tourney_level'] = matches_2025['category'].replace({
    'Grand Slam': "G",
    'ATP Masters 1000': "M",
    'ATP 500': "A",
    'ATP 250': "A",
    'United Cup': "A",    
})

matches_2025['best_of'] = np.where(
    matches_2025['tourney_level'] == 'G', 5, 3
)

matches_2025.rename(columns={
    'tournament_id': 'tourney_id',
    'tournament': 'tourney_name',
    'draw': 'draw_size',
    'player1_id': 'winner_id',
    'player2_id': 'loser_id',
}, inplace=True)

2. Drop irrelevant rounds and tournaments

In [9]:
matches_2025 = matches_2025[matches_2025['round'] != 'ER']
matches_2025 = matches_2025[matches_2025['tourney_name'] != 'United Cup']

### Import Odds Dataset to extract Rank & Points Player Values

In [10]:
odds_2025 = pd.read_excel("./data/odds/2025.xlsx")
odds_2025.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2024-12-29,ATP250,Outdoor,Hard,1st Round,3,Vukic A.,Goffin D.,68,52.0,778,1037.0,6.0,2.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,2.00,1.80,2.08,1.83,2.14,1.83,2.03,1.78
1,1,Brisbane,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Michelsen A.,O Connell C.,41,64.0,1245,795.0,6.0,4.0,4.0,6.0,7.0,6.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.44,2.75,1.48,2.85,1.48,2.90,1.43,2.74
2,1,Brisbane,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Bonzi B.,Tabilo A.,75,23.0,730,1943.0,6.0,7.0,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.67,2.20,1.73,2.22,1.73,2.25,1.67,2.18
3,1,Brisbane,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Nishioka Y.,Rinderknech A.,69,59.0,776,927.0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.53,2.50,1.64,2.39,1.65,2.50,1.59,2.36
4,1,Brisbane,Brisbane International,2024-12-30,ATP250,Outdoor,Hard,1st Round,3,Thompson J.,Berrettini M.,26,34.0,1745,1380.0,3.0,6.0,6.0,3.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,2.63,1.50,2.47,1.60,2.63,1.60,2.48,1.54


1. Assign Player ID to Odds dataset
 - Utilising Ranking dataset to assist accurate matching of players to IDs

In [11]:
rankings = pd.read_csv("./data/rankings/atp_rankings_current.csv")
players = pd.read_csv("./data/players/atp_players.csv")

rankings['ranking_date'] = pd.to_datetime(rankings['ranking_date'], errors='coerce')

# Ascertain most up to date player info
idx_latest = rankings.groupby('player')['ranking_date'].idxmax()
latest_rankings = rankings.loc[idx_latest, ['player', 'rank', 'points']]

# 4. Merge players and rankings dataframes
players = players.merge(
    latest_rankings,
    left_on='player_id',
    right_on='player',
    how='left'
).drop(columns=['player'])

In [13]:
players['rank']    = pd.to_numeric(players['rank'],    errors='coerce')
odds_2025['WRank'] = pd.to_numeric(odds_2025['WRank'], errors='coerce')
odds_2025['LRank'] = pd.to_numeric(odds_2025['LRank'], errors='coerce')

# PLayers Dataframe name formatting for accurate matching
players['last_lower'] = (
    players['name_last']
           .str.lower()
           .str.strip()
)

def make_initials(name):
    if not isinstance(name, str):
        return ""
    parts = name.strip().split()
    initials = [p[0].upper() for p in parts if p]
    return "".join(initials)

players['first_initial'] = players['name_first'].apply(make_initials)

# Handle duplicate player names
dup_counts = (
    players
      .groupby(['last_lower','first_initial'])
      .size()
      .reset_index(name='count')
)
dup_keys = set(zip(
    dup_counts.loc[dup_counts['count']>1, 'last_lower'],
    dup_counts.loc[dup_counts['count']>1, 'first_initial']
))

unique_players = players[
    ~players.set_index(['last_lower','first_initial']).index.isin(dup_keys)
]

id_map = (
    unique_players
      .groupby(['last_lower','first_initial'], as_index=True)['player_id']
      .first()
)

# Odds Dataframe name formatting
def extract_name(df, col_name, prefix):
    """
    Splits df[col_name] on the LAST space:
      - prefix+'_last'       = everything before that space, lowercased
      - prefix+'_first_init' = the token after the space, with dots stripped, uppercased
    """
    raw = df[col_name].fillna("").astype(str)
    parts = raw.str.rsplit(" ", n=1, expand=True)
    
    df[f'{prefix}_last'] = parts[0].str.lower().str.strip()
    df[f'{prefix}_first_init'] = (
        parts[1]
          .str.replace(".", "", regex=False)  # drop all dots
          .str.upper()                        # uppercase the whole string
    )
    return df

odds_2025 = extract_name(odds_2025, 'Winner', 'winner')
odds_2025 = extract_name(odds_2025, 'Loser',  'loser')


# Initial ID assignment via the simple id_map
odds_2025['winner_id'] = id_map.reindex(
    list(zip(odds_2025['winner_last'], odds_2025['winner_first_init']))
).values

odds_2025['loser_id']  = id_map.reindex(
    list(zip(odds_2025['loser_last'], odds_2025['loser_first_init']))
).values


rank_filled = {'winner': set(), 'loser': set()}

# Resolve remaining NaNs by “closest rank”
def resolve_by_rank(df, side):
    id_col   = f'{side}_id'
    last_col = f'{side}_last'
    init_col = f'{side}_first_init'
    rank_col = 'WRank' if side == 'winner' else 'LRank'
    
    mask = df[id_col].isna() & df[rank_col].notna()
    for idx in df.index[mask]:
        lw   = df.at[idx, last_col]
        fi   = df.at[idx, init_col]
        targ = df.at[idx, rank_col]

        cands = players[
            (players['last_lower']    == lw) &
            (players['first_initial'] == fi) &
            players['rank'].notna()
        ].copy()
        if cands.empty:
            continue
        
        cands['diff'] = (cands['rank'] - targ).abs()
        best_idx     = cands['diff'].idxmin()
        best_id      = cands.at[best_idx, 'player_id']
        df.at[idx, id_col] = best_id
        
        rank_filled[side].add(best_id)

resolve_by_rank(odds_2025, 'winner')
resolve_by_rank(odds_2025, 'loser')


# Flag remaining unfilled
both_assigned = rank_filled['winner'].union(rank_filled['loser'])

print(f"Total unique IDs assigned by rank fallback (winner ∪ loser): {len(both_assigned)}")
print("Rows missing winner_id:", odds_2025['winner_id'].isna().sum())
print("Rows missing loser_id: ", odds_2025['loser_id'].isna().sum())


KeyboardInterrupt: 

2. Refactor odds dataset tournament ids and Round values to align with matched dataset

In [ ]:
odds_2025['tourney_id'] = odds_2025['ATP'].replace({
    1 : 339,
    2 : 336,
    3 : 8998,
    4 : 301,
    5 : 580,
    6 : 375,
    7 : 424,
    8: 407,
    9: 506,
    10: 499,
    11: 496,
    12: 451,
    13: 6932,
    14: 807,
    15: 495,
    16: 8996,
    17: 404,
    18: 403,
    19: 4462,
    20: 717,
    21: 360,
    22: 410,
    23: 425,
    24: 308,
    25: 1536,
    26: 416,
    27: 322,
    28: 414,
    29: 520,
    30: 321,
    31: 440,
    32: 500,
    33: 311,
    34: 741,
    35: 8994,
    36: 540,
    37: 316,
    38: 314,
    39: 7480,
    40: 319,
    41: 439,
    42: 418
}
)

In [ ]:
df_tournaments['tournament_id'] = (
    pd.to_numeric(df_tournaments['tournament_id'], errors='coerce')
      .astype('Int64')
)

odds_2025 = odds_2025.merge(
    df_tournaments[['tournament_id','draw']],
    left_on='tourney_id',
    right_on='tournament_id',
    how='left'
)

odds_2025['draw_size'] = (
    odds_2025['draw']
      .str.extract(r'SGL\s*(\d+)', expand=False)
      .astype(int)
)

def code_round(row):
    rd = row['Round']
    ds = row['draw_size']

    for b in (16, 32, 64, 128):
        if ds <= b:
            bracket = b
            break

    m = re.match(r'(\d+)(?:st|nd|rd|th) Round', rd)
    if m:
        n = int(m.group(1))
        size = bracket // (2 ** (n - 1))
        # floor to 16
        if size < 16:
            size = 16
        return f'R{size}'

    if rd == 'Quarterfinals':
        return 'QF'
    if rd == 'Semifinals':
        return 'SF'
    if rd == 'The Final':
        return 'F'
    
    return rd

odds_2025['Round'] = odds_2025.apply(code_round, axis=1)
odds_2025.rename(columns={
    'Round': 'round',
}
, inplace=True)

odds_2025.drop(columns=['tournament_id','draw','draw_size'], inplace=True)


3. Generate Score_Keys to match on for both datasets

In [ ]:
for col in ['W1', 'W2', 'W3', 'W4', 'W5', 'L1', 'L2', 'L3', 'L4', 'L5']:
    odds_2025[col] = odds_2025[col].fillna(0)

# Create score_key by concatenating W1, L1, W2, L2, ..., W5, L5
odds_2025['score_key'] = (
    odds_2025['W1'].astype(int).astype(str) +
    odds_2025['L1'].astype(int).astype(str) +
    odds_2025['W2'].astype(int).astype(str) +
    odds_2025['L2'].astype(int).astype(str) +
    odds_2025['W3'].astype(int).astype(str) +
    odds_2025['L3'].astype(int).astype(str) +
    odds_2025['W4'].astype(int).astype(str) +
    odds_2025['L4'].astype(int).astype(str) +
    odds_2025['W5'].astype(int).astype(str) +
    odds_2025['L5'].astype(int).astype(str)
)
odds_2025.head()

def make_score_key(s):
    if pd.isna(s):
        return None
    s0 = re.sub(r'\([^)]*\)', '', s)
    groups = re.findall(r'\d+', s0)
    key = "".join(groups)
    pad = 5 - len(groups)
    if pad > 0:
        key += "00" * pad
    return key.ljust(10, "0")

matches_2025['score_key'] = matches_2025['score'].apply(make_score_key)

4. Extract Rank & Points Data from Odds DF via multi-step merge
 - Match on unique Tournament, Round, and Score Key triple
 - Resolve duplicate triples via player Ids
 - Final match attempt exclusively on Player Ids

In [ ]:
matches = matches_2025.copy()
odds    = odds_2025.copy()

for df in (matches, odds):
    df['tourney_id'] = pd.to_numeric(df['tourney_id'], errors='coerce').astype('Int64')
    df['score_key']  = pd.to_numeric(df['score_key'],  errors='coerce').astype('Int64')
    df['winner_id']  = pd.to_numeric(df['winner_id'],  errors='coerce').astype('Int64')
    df['loser_id']   = pd.to_numeric(df['loser_id'],   errors='coerce').astype('Int64')
    df['round']      = df['round'].astype(str)

# Triple-key unique merge
odds['ct_triple'] = odds.groupby(['tourney_id','round','score_key'])['score_key'] \
                      .transform('size')

odds_triple = odds[odds['ct_triple'] == 1]

matches = matches.merge(
    odds_triple[['tourney_id','round','score_key','WRank','WPts','LRank','LPts']],
    on=['tourney_id','round','score_key'],
    how='left'
).rename(columns={
    'WRank':'winner_rank',
    'WPts':'winner_points',
    'LRank':'loser_rank',
    'LPts':'loser_points'
})

# Winner-ID fallback
odds['ct_win'] = odds.groupby(['tourney_id','round','winner_id'])['winner_id'] \
                    .transform('size')
odds_win = odds[odds['ct_win'] == 1]

mask = matches['winner_rank'].isna()

merged_win = matches.loc[mask, ['tourney_id','round','score_key','winner_id']] \
    .merge(
        odds_win[['tourney_id','round','score_key','winner_id','WRank','WPts','LRank','LPts']],
        on=['tourney_id','round','score_key','winner_id'],
        how='left'
    )
merged_win.index = matches.index[mask]

matches.loc[mask, 'winner_rank']    = merged_win['WRank']
matches.loc[mask, 'winner_points']  = merged_win['WPts']
matches.loc[mask, 'loser_rank']     = merged_win['LRank']
matches.loc[mask, 'loser_points']   = merged_win['LPts']

# Loser-ID fallback
odds['ct_los'] = odds.groupby(['tourney_id','round','loser_id'])['loser_id'] \
                    .transform('size')
odds_los = odds[odds['ct_los'] == 1]

mask = matches['winner_rank'].isna()

merged_los = matches.loc[mask, ['tourney_id','round','score_key','loser_id']] \
    .merge(
        odds_los[['tourney_id','round','score_key','loser_id','WRank','WPts','LRank','LPts']],
        on=['tourney_id','round','score_key','loser_id'],
        how='left'
    )
merged_los.index = matches.index[mask]

matches.loc[mask, 'winner_rank']    = merged_los['WRank']
matches.loc[mask, 'winner_points']  = merged_los['WPts']
matches.loc[mask, 'loser_rank']     = merged_los['LRank']
matches.loc[mask, 'loser_points']   = merged_los['LPts']
matches.drop(columns=['year', 'player1_seed', 'player2_seed', 'url', 'court_vision', 'end'], inplace=True)

### Scrape Match Stats from Infosys API (utilising infotennis repo)
1. Extract stats match by match from returned json data

In [ ]:
#United Cup + 4 Grand Slams
SKIP_TOURNAMENTS = {"9900", "580", "520", "540", "560"}

# JSON Stats tag keys
SIMPLE_COUNT_STATS = {
    "Aces":                   "ace",
    "Double Faults":          "df",
    "Service Games Played":   "SvGms",
}

RATIO_STATS = {
    "1st Serve":                "1stIn",
    "1st Serve Points Won":     "1stWon",
    "2nd Serve Points Won":     "2ndWon",
    "Service Points Won":       "svpt",
    "Break Points Saved":       "bpSaved",
}

_frac_re = re.compile(r"(\d+)\/(\d+)")

# ── HELPERS ────────────────────────────────────────────────────────────────────

def _aggregate_set_counts(raw: dict) -> dict:
    """
    Pull only the overall‐match stats block ('set0' if present, else first block)
    and return a dict of all your count‐based stats.
    """
    # Initialise Stats
    agg = {}
    for short in SIMPLE_COUNT_STATS.values():
        agg[f"p1_{short}"] = 0
        agg[f"p2_{short}"] = 0
    for short in RATIO_STATS.values():
        agg[f"p1_{short}"] = 0
        agg[f"p2_{short}"] = 0
        if short == "bpSaved":
            agg[f"p1_{short}Faced"] = 0
            agg[f"p2_{short}Faced"] = 0

    # Get Stats and Process
    set_stats = raw.get("setStats", {})
    if "set0" in set_stats:
        stats_list = set_stats["set0"]
    else:
        stats_list = next(iter(set_stats.values()), [])

    for stat in stats_list:
        name, p1, p2 = stat["name"], stat["player1"], stat["player2"]

        if name in SIMPLE_COUNT_STATS:
            key = SIMPLE_COUNT_STATS[name]
            agg[f"p1_{key}"] += int(p1)
            agg[f"p2_{key}"] += int(p2)

        elif name in RATIO_STATS:
            key = RATIO_STATS[name]
            m1 = _frac_re.match(p1)
            m2 = _frac_re.match(p2)
            if m1:
                n1, d1 = map(int, m1.groups())
                agg[f"p1_{key}"] += n1
                if key == "bpSaved":
                    agg[f"p1_{key}Faced"] += d1
            if m2:
                n2, d2 = map(int, m2.groups())
                agg[f"p2_{key}"] += n2
                if key == "bpSaved":
                    agg[f"p2_{key}Faced"] += d2

    # Output dict format
    out = {}
    for display, short in SIMPLE_COUNT_STATS.items():
        out[f"player1_{short}"] = agg[f"p1_{short}"]
        out[f"player2_{short}"] = agg[f"p2_{short}"]

    for display, short in RATIO_STATS.items():
        out[f"player1_{short}"] = agg[f"p1_{short}"]
        out[f"player2_{short}"] = agg[f"p2_{short}"]
        if short == "bpSaved":
            out["player1_bpFaced"] = agg[f"p1_{short}Faced"]
            out["player2_bpFaced"] = agg[f"p2_{short}Faced"]

    return out

# ── MAIN SCRAPER ───────────────────────────────────────────────────────────────

def scrape_and_append_key_stats(
    matches_df: pd.DataFrame,
    year: int = 2025
) -> (pd.DataFrame, pd.DataFrame):
    """
    For each match in matches_df (with columns
      ['tourney_id','match_id','player1_sysid','player2_sysid']),
    fetch the JSON key-stats, aggregate them, and assign each JSON
    player’s stats to the correct side based on sysid match.
    Returns:
      - merged_df: original matches_df plus player1_*/player2_* stats
      - failed_df: subset of rows where neither JSON ID matched
    """
    
    results = []
    failures = []
    decode_failures = 0

    subset = matches_df.copy()
    for _, row in tqdm(subset.iterrows(),
                       total=len(subset),
                       desc="Scraping key-stats"):
        tourn_id = row["tourney_id"]
        match_id = row["match_id"]
        sys1     = str(row["player1_sysid"]).strip().upper()
        sys2     = str(row["player2_sysid"]).strip().upper()

        # Fetch JSON
        raw = None
        for _ in range(3):
            try:
                raw = scrape_ATP_match_data(year, str(tourn_id), match_id, "key-stats")
                break
            except JSONDecodeError:
                time.sleep(1)
        if raw is None:
            decode_failures += 1
            failures.append({
                **row.to_dict(),
                **{f"player{i}_{k}": None
                   for i in (1,2)
                   for k in list(SIMPLE_COUNT_STATS.values())+list(RATIO_STATS.values())}
            })
            continue

        # Extract JSON player blocks and IDs
        players_json = raw.get("players", [])
        if len(players_json) < 2:
            failures.append({**row.to_dict(), **{}})
            continue

        j1 = str(players_json[0].get("player1Id")).strip().upper()
        j2 = str(players_json[1].get("player1Id")).strip().upper()

        # Map Stats to correct player based on ID
        mapping = {}
        mapping["player1"] = ("player1" if j1 == sys1 else
                              "player2" if j1 == sys2 else None)
        mapping["player2"] = ("player1" if j2 == sys1 else
                              "player2" if j2 == sys2 else None)

        stats = _aggregate_set_counts(raw)

        remapped = {}
        for json_side in ("player1","player2"):
            match_side = mapping[json_side]
            for key, val in stats.items():
                prefix, metric = key.split("_",1)
                if prefix == json_side:
                    if match_side is None:
                        remapped[f"{json_side}_{metric}"] = None
                    else:
                        remapped[f"{match_side}_{metric}"] = val

        # Output rows formatting
        out = {
            "tourney_id":    tourn_id,
            "match_id":      match_id,
            "player1_sysid": sys1,
            "player2_sysid": sys2,
        }

        all_metrics = list(SIMPLE_COUNT_STATS.values()) + list(RATIO_STATS.values())
        for m in all_metrics:
            out.setdefault(f"player1_{m}", None)
            out.setdefault(f"player2_{m}", None)

        out.update(remapped)
        results.append(out)

        # Record failed match stat scrapes
        if mapping["player1"] is None and mapping["player2"] is None:
            failures.append(out)

        time.sleep(0.2)

    if decode_failures:
        print(f"⚠️ {decode_failures} JSON decode failures")

    # Assemble DataFrames
    stats_df  = pd.DataFrame(results)
    failed_df = pd.DataFrame(failures).drop_duplicates(subset=["tourney_id","match_id"])

    merged = matches_df.merge(
        stats_df,
        on=["tourney_id","match_id"],
        how="left"
    )
    return merged, failed_df


In [ ]:
matches_save = matches.copy()
full, fail = scrape_and_append_key_stats(matches)
full.to_csv("./data/2025/postScrapeDataset.csv", index=False)
full.head()

Scraping key-stats: 100%|██████████| 1989/1989 [36:34<00:00,  1.10s/it] 

⚠️ 501 JSON decode failures


,tourney_name,tourney_id,round,player1_name,player1_sysid_x,player1_nation,player2_name,player2_sysid_x,player2_nation,score,match_id,tourney_date,category,draw_size,surface,winner_id,loser_id,player1_height,player2_height,player1_age,player2_age,tourney_level,best_of,score_key,winner_rank,winner_points,loser_rank,loser_points,player1_sysid_y,player2_sysid_y,player1_ace,player2_ace,player1_df,player2_df,player1_SvGms,player2_SvGms,player1_1stIn,player2_1stIn,player1_1stWon,player2_1stWon,player1_2ndWon,player2_2ndWon,player1_svpt,player2_svpt,player1_bpSaved,player2_bpSaved,player1_bpFaced,player2_bpFaced
0,Brisbane International presented by Evie,339,F,Jiri Lehecka,l0bv,cze,Reilly Opelka,o522,usa,41,ms001,2024-12-29,ATP 250,32.0,Hard,208103,124187,185.0,211.0,23.1,27.3,A,3,4100000000,28.0,1660.0,293.0,176.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brisbane International presented by Evie,339,SF,Jiri Lehecka,l0bv,cze,Grigor Dimitrov,d875,bul,64 44,ms003,2024-12-29,ATP 250,32.0,Hard,208103,105777,185.0,191.0,23.1,33.6,A,3,6444000000,28.0,1660.0,10.0,3350.0,L0BV,D875,4.0,6.0,3.0,2.0,9.0,9.0,26.0,32.0,22.0,24.0,17.0,12.0,39.0,36.0,0.0,1.0,0.0,2.0
2,Brisbane International presented by Evie,339,SF,Reilly Opelka,o522,usa,Giovanni Mpetshi Perricard,m0gz,fra,63 76(4),ms002,2024-12-29,ATP 250,32.0,Hard,124187,208659,211.0,203.0,27.3,21.5,A,3,6376000000,293.0,176.0,31.0,1561.0,O522,M0GZ,12.0,10.0,2.0,5.0,11.0,10.0,45.0,48.0,38.0,38.0,13.0,9.0,51.0,47.0,4.0,1.0,4.0,2.0
3,Brisbane International presented by Evie,339,QF,Reilly Opelka,o522,usa,Novak Djokovic,d643,srb,76(6) 63,ms004,2024-12-29,ATP 250,32.0,Hard,124187,104925,211.0,188.0,27.3,37.6,A,3,7663000000,293.0,176.0,7.0,3910.0,O522,D643,16.0,8.0,1.0,1.0,11.0,10.0,49.0,53.0,38.0,38.0,13.0,12.0,51.0,50.0,1.0,4.0,1.0,5.0
4,Brisbane International presented by Evie,339,QF,Grigor Dimitrov,d875,bul,Jordan Thompson,tc61,aus,61 21,ms007,2024-12-29,ATP 250,32.0,Hard,105777,111442,191.0,183.0,33.6,30.7,A,3,6121000000,10.0,3350.0,26.0,1745.0,D875,TC61,1.0,1.0,1.0,1.0,5.0,5.0,21.0,12.0,19.0,7.0,5.0,6.0,24.0,13.0,0.0,1.0,0.0,4.0


2. Refactor Match Stats Columns

In [ ]:
main_2025 = full.rename(
    columns={col: col.replace('player1_', 'winner_').replace('player2_', 'loser_')
             for col in full.columns if col.startswith('player1_') or col.startswith('player2_')}
)

In [ ]:
main_2025 = main_2025.drop(columns=['winner_sysid_x', 'loser_sysid_x', 'winner_sysid_y', 'loser_sysid_y', 'match_id', 'category', 'score_key'])
main_2025 = main_2025.rename(columns={
    'match_id': 'match_num',
    'winner_height': 'winner_ht',
    'loser_height': 'loser_ht',
    'winner_nation': 'winner_ioc',
    'loser_nation': 'loser_ioc',
    'winner_points': 'winner_rank_points',
    'loser_points': 'loser_rank_points',
    'winner_ace': 'w_ace',
    'loser_ace': 'l_ace',
    'winner_df': 'w_df',
    'loser_df': 'l_df',
    'winner_svpt': 'w_svpt',
    'loser_svpt': 'l_svpt',
    'winner_1stIn': 'w_1stIn',
    'loser_1stIn': 'l_1stIn',
    'winner_1stWon': 'w_1stWon',
    'loser_1stWon': 'l_1stWon',
    'winner_2ndWon': 'w_2ndWon',
    'loser_2ndWon': 'l_2ndWon',
    'winner_bpSaved': 'w_bpSaved',
    'loser_bpSaved': 'l_bpSaved',
    'winner_bpFaced': 'w_bpFaced',
    'loser_bpFaced': 'l_bpFaced',
    'winner_SvGms': 'w_SvGms',
    'loser_SvGms': 'l_SvGms',
})

3. Refactor tournament names to align with Jeff Sackmann datasets

In [ ]:
tourney_name_map = {
    339: "Brisbane",
    336: "Hong Kong",
    8898: "Adelaide",
    301: "Auckland",
    580: "Australian Open",
    375: "Montpellier",
    424: "Dallas",
    407: "Rotterdam",
    496: "Marseille",
    499: "Delray Beach",
    506: "Buenos Aires",
    451: "Doha",
    6932: "Rio De Janeiro",
    807: "Acapulco",
    8996: "Santiago",
    404: "Indian Wells Masters",
    403: "Miami Masters",
    717: "Houston",
    360: "Marrakech",
    410: "Monte Carlo Masters",
    425: "Barcelona",
    308: "Munich",
    1526: "Madrid Masters",
    416: "Rome Masters",
    414: "Hamburg",
    322: "Geneva",
    520: "Roland Garros",
    321: "Stuttgart",
    440: "s Hertogenbosch",
    311: "Queen's Club",
    500: "Halle",
    8994: "Mallorca",
    741: "Eastbourne",
    540: "Wimbledon",
    7480: "Los Cabos",
    314: "Gstaad",
    316: "Bastad",
    439: "Umag",
    418: "Washington",
    319: "Kitzbuhel",
    421: "Canada Masters"
}

full['tourney_name'] = full['tourney_id'].map(tourney_name_map).fillna(full['tourney_name'])
main_2025['tourney_name'] = main_2025['tourney_id'].map(tourney_name_map).fillna(main_2025['tourney_name'])
main_2025 = main_2025.dropna(subset=['winner_id', 'loser_id'])

In [ ]:
def reformat_score(score):
    """
    Reformat tennis scores by adding dashes between set scores.
    Example: '41 46 76(4)' -> '4-1 4-6 7-6(4)'
    """
    formatted_score = re.sub(r'(\d)(\d)(\(\d+\))?', r'\1-\2\3', score)
    return formatted_score

main_2025['score'] = main_2025['score'].apply(reformat_score)

### Export Final 2025 Matches Dataset

In [ ]:
main_2025.to_csv("./data/all/atp_matches_scrape_2025.csv")
main_2025.head()

,tourney_name,tourney_id,round,winner_name,winner_ioc,loser_name,loser_ioc,score,tourney_date,draw_size,surface,winner_id,loser_id,winner_ht,loser_ht,winner_age,loser_age,tourney_level,best_of,winner_rank,winner_rank_points,loser_rank,loser_rank_points,w_ace,l_ace,w_df,l_df,w_SvGms,l_SvGms,w_1stIn,l_1stIn,w_1stWon,l_1stWon,w_2ndWon,l_2ndWon,w_svpt,l_svpt,w_bpSaved,l_bpSaved,w_bpFaced,l_bpFaced
0,Brisbane,339,F,Jiri Lehecka,cze,Reilly Opelka,usa,4-1,2024-12-29,32.0,Hard,208103,124187,185.0,211.0,23.1,27.3,A,3,28.0,1660.0,293.0,176.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brisbane,339,SF,Jiri Lehecka,cze,Grigor Dimitrov,bul,6-4 4-4,2024-12-29,32.0,Hard,208103,105777,185.0,191.0,23.1,33.6,A,3,28.0,1660.0,10.0,3350.0,4.0,6.0,3.0,2.0,9.0,9.0,26.0,32.0,22.0,24.0,17.0,12.0,39.0,36.0,0.0,1.0,0.0,2.0
2,Brisbane,339,SF,Reilly Opelka,usa,Giovanni Mpetshi Perricard,fra,6-3 7-6(4),2024-12-29,32.0,Hard,124187,208659,211.0,203.0,27.3,21.5,A,3,293.0,176.0,31.0,1561.0,12.0,10.0,2.0,5.0,11.0,10.0,45.0,48.0,38.0,38.0,13.0,9.0,51.0,47.0,4.0,1.0,4.0,2.0
3,Brisbane,339,QF,Reilly Opelka,usa,Novak Djokovic,srb,7-6(6) 6-3,2024-12-29,32.0,Hard,124187,104925,211.0,188.0,27.3,37.6,A,3,293.0,176.0,7.0,3910.0,16.0,8.0,1.0,1.0,11.0,10.0,49.0,53.0,38.0,38.0,13.0,12.0,51.0,50.0,1.0,4.0,1.0,5.0
4,Brisbane,339,QF,Grigor Dimitrov,bul,Jordan Thompson,aus,6-1 2-1,2024-12-29,32.0,Hard,105777,111442,191.0,183.0,33.6,30.7,A,3,10.0,3350.0,26.0,1745.0,1.0,1.0,1.0,1.0,5.0,5.0,21.0,12.0,19.0,7.0,5.0,6.0,24.0,13.0,0.0,1.0,0.0,4.0
